# PigeonXT - Quickly annotate data in Jupyter Lab

#### [Dennis Bakhuis](https://linkedin.com/in/dennisbakhuis/) - 28th May 2020
https://github.com/dennisbakhuis/pigeonXT

PigeonXT is an extention to the original [Pigeon](https://github.com/agermanidis/pigeon), created by [Anastasis Germanidis](https://pypi.org/user/agermanidis/).
PigeonXT is a simple widget that lets you quickly annotate a dataset of
unlabeled examples from the comfort of your Jupyter notebook.

Pigeon currently support the following annotation tasks:
- binary / multi-class classification
- multi-label classification
- regression tasks
- captioning tasks

Anything that can be displayed on Jupyter
(text, images, audio, graphs, etc.) can be displayed by pigeon
by providing the appropriate :code:`display_fn` argument.

Additionally, custom hooks can be attached to each row update (:code:`example_process_fn`),
or when the annotating task is complete(:code:`final_process_fn`).

## Installation
PigeonXT obviously needs a Jupyter Lab environment. Futhermore, it requires ipywidgets.
The widget itself can be installed using pip:
```bash
    pip install pigeonXT-jupyter
```

To run the provided examples in a new environment using Conda:
```bash
    conda create --name pigeon python=3.7
    conda activate pigeon
    conda install nodejs
    pip install numpy pandas jupyterlab ipywidgets
    pip install pigeonXT-jupyter
```

Starting Jupyter Lab environment:
```bash
    jupyter lab
```


## Binary or multi-class classification

In [1]:
from pigeonXT import annotate

In [2]:
annotations = annotate(
  ['I love this movie', 'I was really disappointed by the book'],
  options=['positive', 'negative', 'inbetween']
)

HTML(value='0 of 2 Examples annotated, Current Position: -1 ')

Output()

In [3]:
annotations

{}

## Multi-label classification

In [1]:
from pigeonXT import annotate
import pandas as pd

In [4]:
df = pd.DataFrame([
    {'title': 'Star wars'},    
    {'title': 'The Positively True Adventures of the Alleged Texas Cheerleader-Murdering Mom'},
    {'title': 'Eternal Sunshine of the Spotless Mind'},
    {'title': 'Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb'},    
    {'title': 'Killer klowns from outer space'},    
])

labels = ['Adventure', 'Romance', 'Fantasy', 'Science fiction', 'Horror', 'Thriller']

In [5]:
annotations = annotate( df.title, 
                      options=labels, 
                      task_type='multilabel-classification',
                      buttons_in_a_row=3,
                      reset_buttons_after_click=True,
                      include_skip=True,
                      include_back=True)     


HTML(value='0 of 5 Examples annotated, Current Position: -1 ')

Output()

In [6]:
annotations

{'Star wars': ['Science fiction'],
 'The Positively True Adventures of the Alleged Texas Cheerleader-Murdering Mom': ['Science fiction']}

## Image labeling

In [8]:
from pigeonXT import annotate
from IPython.display import display, Image

In [9]:
annotations = annotate(
  ['assets/img_example1.jpg', 'assets/img_example2.jpg'],
  options=['cat', 'dog', 'horse'],
  display_fn=lambda filename: display(Image(filename))
)

HTML(value='0 examples annotated, 2 examples left')

Output()

In [10]:
annotations

{'assets/img_example1.jpg': 'cat'}

## Multi-label classification with custom hooks

In [11]:
from pigeonXT import annotate
import pandas as pd
import numpy as np
from pathlib import Path

In [12]:
df = pd.DataFrame([
    {'title': 'Star wars'},    
    {'title': 'The Positively True Adventures of the Alleged Texas Cheerleader-Murdering Mom'},
    {'title': 'Eternal Sunshine of the Spotless Mind'},
    {'title': 'Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb'},    
    {'title': 'Killer klowns from outer space'},    
])

labels = ['Adventure', 'Romance', 'Fantasy', 'Science fiction', 'Horror', 'Thriller']
shortLabels = ['A', 'R', 'F', 'SF', 'H', 'T']

df.to_csv('inputtestdata.csv', index=False)

In [17]:
def setLabels(labels, numClasses):
    row = np.zeros([numClasses], dtype=np.uint8)
    row[labels] = 1
    return row

def labelPortion(inputFile, 
                 labels = ['yes', 'no'], 
                 outputFile='output.csv', 
                 portionSize=2,
                 textColumn='title',
                 shortLabels=None):
    if shortLabels == None:
        shortLabels = labels
    out = Path(outputFile)
    if out.exists():
        outdf = pd.read_csv(out)
        currentId = outdf.index.max() + 1
    else:
        currentId = 0
    indf = pd.read_csv(inputFile)
    examplesInFile = len(indf)
    indf = indf.loc[currentId:currentId + portionSize - 1]
    actualPortionSize = len(indf)
    print(f'{currentId + 1} - {currentId + actualPortionSize} of {examplesInFile}')
    sentences = indf[textColumn].tolist()
    
    for label in shortLabels:
        indf[label] = None
    
    def updateRow(title, selectedLabels):
        print(title, selectedLabels)
        labs = setLabels([labels.index(y) for y in selectedLabels], len(labels))
        indf.loc[indf.title == title, shortLabels] = labs
    
    def finalProcessing(annotations):
        if out.exists():
            prevdata = pd.read_csv(out)
            outdata = pd.concat([prevdata, indf]).reset_index(drop=True)
        else:
            outdata = indf.copy()
        outdata.to_csv(out, index=False)
    
    annotated = annotate( sentences, 
                          options=labels, 
                          task_type='multilabel-classification',
                          buttons_in_a_row=3,
                          reset_buttons_after_click=True,
                          include_skip=False,
                          example_process_fn=updateRow,
                          final_process_fn=finalProcessing)     
    return indf

In [18]:
annotations = labelPortion('inputtestdata.csv',
                           labels=labels,
                           shortLabels= shortLabels)

1 - 2 of 5


HTML(value='0 examples annotated, 2 examples left')

Output()

In [19]:
annotations # check while still annotating

,title,A,R,F,SF,H,T
0,Star wars,1,0,0,1,0,0
1,The Positively True Adventures of the Alleged ...,0,1,0,0,1,0


In [20]:
annotations

,title,A,R,F,SF,H,T
0,Star wars,1,0,0,1,0,0
1,The Positively True Adventures of the Alleged ...,0,1,0,0,1,0


In [21]:
annotations = labelPortion('inputtestdata.csv',
                           labels=labels,
                           shortLabels= shortLabels)

3 - 4 of 5


HTML(value='0 examples annotated, 2 examples left')

Output()

In [22]:
annotations

,title,A,R,F,SF,H,T
2,Eternal Sunshine of the Spotless Mind,None,None,None,None,None,None
3,Dr. Strangelove or: How I Learned to Stop Worr...,None,None,None,None,None,None


In [23]:
annotations = labelPortion('inputtestdata.csv',
                           labels=labels,
                           shortLabels= shortLabels)

3 - 4 of 5


HTML(value='0 examples annotated, 2 examples left')

Output()

In [24]:
annotations

,title,A,R,F,SF,H,T
2,Eternal Sunshine of the Spotless Mind,None,None,None,None,None,None
3,Dr. Strangelove or: How I Learned to Stop Worr...,None,None,None,None,None,None


In [25]:
out = pd.read_csv('output.csv')
out

,title,A,R,F,SF,H,T
0,Star wars,1,0,0,1,0,0
1,The Positively True Adventures of the Alleged ...,0,1,0,0,1,0
